# A Toolbox for Solving Dynamic Rational Inattention Problems

This notebook goes through a couple of examples for solving LQG Dynamic Rational Inattention models using the methods and the [solver](https://github.com/choongryulyang/dynamic_multivariate_RI) from [Afrouzi and Yang (2019)](http://www.afrouzi.com/dynamic_inattention.pdf) that was adapted for Julia by [Miguel Acosta](https://www.acostamiguel.com/home). To include the solver, execute:

In [ ]:
include("Solve_RI_Dynamics.jl")

## A Structure for LQG Dynamic Rational Inattention Problems (drip)

### The Problem
A LQG Dynamic Rational Inattention Problem (**drip**) is defined as the following tracking problem, where at any point in time the agent chooses a vector of actions $\vec{a}_t\in\mathbb{R}^m$ to track a Gaussian stochastic process $\vec{x}_t\in\mathbb{R}^n$:
\begin{align}
    & \min_{\{\vec{a}_t\}_{t\geq 0}} \mathbb{E}\left[\sum_{t=0}^\infty \beta^t (\vec{a}_t - \vec{x}_t'\mathbf{H})'(\vec{a}_t - \vec{x}_t'\mathbf{H}) - \omega \mathbb{I}(\vec{a}^t;\vec{x}^t|\vec{a}^{t-1})\lvert \vec{a}^{-1}\right] \\
    s.t.\quad &
        \vec{x}_t=\mathbf{A}\vec{x}_{t-1}+\mathbf{Q}\vec{u}_t,\quad \vec{u}_t\sim \mathcal{N}(\mathbf{0},\mathbf{I}^{k\times k})
\end{align}
Here:
* $\vec{a}_t\in\mathbb{R}^m$ is the vector of the agent's actions at time $t$ (a firms choosing a price, or a househld choosing consumption and labor etc.) We denote the number of actions by $m$.
* $\vec{x}_t\in\mathbb{R}^n$ is the vector of the shocks that the agent faces at time $t$ that are exogenous to her decision, but could be endogenous to the GE model (marginal cost of a firm, real interest rates etc.) We denote the number of shocks by $n$.

#### The Parameters
The LQG-DRI problem is characterized by the following parameters:
* $\omega\in \mathbb{R}_+$: cost of 1 bit of information in units of the agent's payoff. 
* $\beta\in[0,1]$: rate of discounting information.
* $\mathbf{A}\in \mathbb{R}^{n\times n}, \mathbf{Q}\in\mathbb{R}^{n\times k}$: Determine the state space representation of $\vec{x}_t$.
* $\mathbf{H}\in \mathbb{R}^{n\times m}$: interaction of payoffs with shocks. This comes from a second order approximation to the utility function and is such that under full information $\vec{a}^*=\mathbf{H}'\vec{x}$.

#### The Solution
The solution to the dynamic rational inattention problem is a joint stochastic process between the actions and the states: $\{(\vec{a}_t,\vec{x}_t):t\geq 0\}$. Moreover, in some economic applications, we are also interested in the law of motion for the agent's belief about $\vec{x}_t$ under the optimal information structure $\hat{x}_t=\mathbb{E}_t[\vec{x}_t]$ where the expectation is taken conditional on the agent's time $t$ information.

Theorem 2 and Proposition 3 in [Afrouzi and Yang (2019)](http://www.afrouzi.com/dynamic_inattention.pdf) characterize this joint distribution as a function of a tuple $(\mathbf{K},\mathbf{Y},\mathbf{\Sigma}_z)$ where
\begin{align}
    \vec{a}_t &= \mathbf{H}'\hat{x}_t = \mathbf{H}'\mathbf{A}\hat{x}_{t-1}+\mathbf{Y}'(\vec{x}_t-\mathbf{A}\hat{x}_{t-1})+\vec{z}_t \\
    \hat{x}_t &= \mathbf{A}\hat{x}_{t-1}+\mathbf{K}\mathbf{Y}'(\vec{x}_t-\mathbf{A}\hat{x}_{t-1})+\mathbf{K}\vec{z}_t,\quad \vec{z}\sim\mathcal{N}(0,\Sigma_z)
\end{align}

Here, 

* $\mathbf{K}\in\mathbb{R}^{n\times m}$ is the Kalman-gain matrix of the agent in under optimal information acquisition in the stationary solution.
* $\mathbf{Y}\in\mathbb{R}^{m\times m}$ maps the state to the agent's action under rational inattetion and govern the covariance of the two.
* $\mathbf{\Sigma}_z\in\mathbb{R}^{m\times m}$ is the variance-covariance matrix of the agent's rational inattention error.

In addition to these, we might also be interested in the agent's prior and posterior subjective undertainty, along with the continution value that she assigns to information:
* $\mathbf{\Sigma}_{p}=\mathbb{V}ar(\vec{x}_t|\vec{a}^{t})\in\mathbb{R}^{n\times n}$.
* $\mathbf{\Sigma}_{-1}=\mathbb{V}ar(\vec{x}_t|\vec{a}^{t-1})\in\mathbb{R}^{n\times n}$, 
* $\bar{\Omega}\in\mathbb{R}^{n\times n}$.

The matrix $\bar{\Omega}$, which is the continuation value of information, is also important for the number of iterations that the code needs to converge. When $n$ is large, accurate guesses for $\bar{\Omega}$ and $\mathbf{\Sigma}_{-1}$ speed up the convergence considerably.

### The Solver

The solver function is `Solve_RI_Dynamics(ω,β,A,Q,H;Σ0,Ω0)`. It takes the primitives `(ω,β,A,Q,H;Σ0,Ω0)` as arguments, where the arguments `Σ0,Ω0` are the initial guesses for $\mathbf{\Sigma}_{-1}$ and $\bar{{\Omega}}$ and are optional for cases in which an accurate initial guess is at hand. If not specified, the default values are set as `Ω0=H*H'` and `Σ0=Q*Q'`. 

The function returns the solution of the model within a `drip` structure,  that contains all the primitives and the solution objects of the model:

In [ ]:
struct drip
    ω; β; A; Q; H;          #Primitives
    K; Y; Σ_z; Σ_p; Σ_1; Ω; #Solution
end

### Impulse Resonse Functions

Once the model is solved, one can generate the impulse response functions of actions and beliefs using the laws of motion stated above. 

We have also included a built-in function that generates these IRFs. The function `dripirfs(P::drip,T::Int)` takes a `drip` structure as input along with the length of irfs `T` and returns the irfs of the state, beliefs and actions to all possible shocks in the following structure:

In [ ]:
struct dripirfs
    T::Int                  # Length of IRFs
    x::Array{Float64}       # IRFs of the state
    x_hat::Array{Float64}   # IRFs of beliefs
    a::Array{Float64}       # IRFs of actions
end

Here 
* `x` is of dimension $n\times k \times T$ where `x[i,j,:]` is the IRF of the $i$'th element of the state vector ($\vec{x}_t$) to the $j$'th shock in the vector $\vec{u}_t$.

* `x_hat` is of dimension $n\times k \times T$ where `x_hat[i,j,:]` is the IRF of the agent's belief about $i$'th element of the state vector  ($\hat{x}_t$) to the $j$'th shock in the vector $\vec{u}_t$.

* `x_hat` is of dimension $m\times k \times T$ where `a[i,j,:]` is the IRF of the $i$'th element of the action vector ($\vec{a}_t$) to the $j$'th shock in the vector $\vec{u}_t$.

## Example 1: Pricing in Partial Equilibrium
There is a measure of firms indexed by $i\in[0,1]$. Firm $i$ chooses its price $p_{i,t}$ at time $t$ to track its ideal price $p_{i,t}^*$. Formally, her flow profit is  
    $$-(p_{i,t}-p_{i,t}^*)^2$$
    
We start by assuming that $p_{i,t}^*=q_t$ where
\begin{align}
    \Delta q_t&=\rho \Delta q_{t-1}+u_t,\quad u_t\sim \mathcal{N}(0,\sigma_u^2) 
\end{align}
Here $q_t$ can be interpreted as money growth or the nominal aggregate demand. Therefore, the state-space representation of the problem is 
\begin{align}
   \vec{x}_{t}&=\left[\begin{array}{c}
    q_{t}\\
    \Delta q_{t}
    \end{array}\right] 
    = \underset{\mathbf{A}}{\underbrace{\left[\begin{array}{cc}
    1 & \rho \\
    0 & \rho \\
    \end{array}\right]}}\, \vec{x}_{t-1} 
    + \underset{\mathbf{Q}}{\underbrace{\left[\begin{array}{c}
    \sigma_u  \\
    \sigma_u \\
    \end{array}\right]}}\, u_t, \\
    p_{i,t}^*&=\underset{\mathbf{H}}{\underbrace{\left[\begin{array}{c}
    1 \\
    0 \\
    \end{array}\right]}}'\vec{x}_t
\end{align}

Assign value to deep parameters and define the structure of the problem

In [ ]:
ρ   = 0.6;        #persistence of money growth
σ_u = 1;          #std. deviation of shocks to money growth

## specifying the primitives of the drip
ω   = 100;       
β   = 0.96^0.25;  
A   = [1 ρ; 0 ρ]; 
Q   = σ_u*[1; 1];
H   = [1; 0];

#### Solve

In [ ]:
ex1  = Solve_RI_Dynamics(ω,β,A,Q,H);

Get IRFs:

In [ ]:
ex1irfs = dripirfs(ex1,20);

Let's plot how the average price $p=\int_0^1 p_{i,t}di$ responds to a shock to money growth:

In [ ]:
using Plots, LaTeXStrings; gr()
plot(1:ex1irfs.T,[ex1irfs.x[1,1,:],ex1irfs.a[1,1,:]],
    xlabel     = "Time",
    label      = ["Nominal Agg. Demand (q)" "Price (p)"],
    title      = "IRFs to 1 Std. Dev. Expansionary Shock",
    xlim       = (1,ex1irfs.T),
    lw         = 3,
    legend     = :bottomright,
    legendfont = font(10),
    tickfont   = font(10),
    framestyle = :box)

We can also plot the IRFs of inflation $\pi_t\equiv p_t-p_{t-1}$ and output $y_t\equiv q_t-p_t$ to 1 percent expansionary shock to $q$:

In [ ]:
p1 = plot(1:ex1irfs.T,ex1irfs.x[1,1,:]-ex1irfs.a[1,1,:],
    title  = "Output (y)")

p2 = plot(1:ex1irfs.T,[ex1irfs.a[1,1,1];ex1irfs.a[1,1,2:end]-ex1irfs.a[1,1,1:end-1]],
    title  = "Inflation (pi)")

display(plot(p1,p2,
    layout     = (1,2),
    xlim       = (1,ex1irfs.T),
    lw         = 3,
    legend     = false,
    tickfont   = font(10),
    framestyle = :box));

## Example 2: Pricing in Partial Equilibrium with Transitory Shocks
Suppose now that $p_{i,t}^*=q_t-z_{t}$ where
\begin{align}
    \Delta q_t&=\rho \Delta q_{t-1}+u_t,\quad u_t\sim \mathcal{N}(0,\sigma_u^2) \\
    z_{t}&\sim \mathcal{N}(0,\sigma_z^2) 
\end{align}
Here $q_t$ can be interpreted as money growth and $z_{i,t}$ as an idiosyncratic TFP shock. Therefore, 
\begin{align}
   \vec{x}_{t}&=\left[\begin{array}{c}
    q_{t}\\
    \Delta q_{t} \\
    z_{t}
    \end{array}\right] 
    = \underset{\mathbf{A}}{\underbrace{\left[\begin{array}{ccc}
    1 & \rho & 0\\
    0 & \rho & 0\\
    0 & 0 & 0 \\
    \end{array}\right]}}\, \vec{x}_{t-1} 
    + \underset{\mathbf{Q}}{\underbrace{\left[\begin{array}{cc}
    \sigma_u & 0 \\
    \sigma_u & 0 \\
    0 & \sigma_z   \\
    \end{array}\right]}}\, \left[\begin{array}{c}
    u_t  \\
    z_{t}   \\
    \end{array}\right], \\
    p_{i,t}^*&=\underset{\mathbf{H}}{\underbrace{\left[\begin{array}{c}
    1     \\
    0 \\
    -1 \\
    \end{array}\right]}}'\vec{x}_{t}
\end{align}

Assign values:

In [ ]:
ρ   = 0.6;        #persistence of money growth
σ_u = 1;          #std. deviation of shocks to money growth
σ_z = √10;      #std. deviation of idiosyncratic shock

## specifying the primitives of the drip
ω   = 100;       
β   = 0.96^0.25;  
A   = [1 ρ 0; 0 ρ 0; 0 0 0]; 
Q   = [σ_u 0; σ_u 0; 0 σ_z];
H   = [1; 0; -1];

#### Solve

In [ ]:
ex2  = Solve_RI_Dynamics(ω,β,A,Q,H);

Get the IRFs:

In [ ]:
ex2irfs = dripirfs(ex2,20);

To get the IRFs simply use the law of motion for actions:

In [ ]:
p1 = plot(1:ex2irfs.T,[ex2irfs.x[1,1,:],ex2irfs.a[1,1,:]],
    title  = "IRFs to q shock");
p2 = plot(1:ex1irfs.T,[ex2irfs.x[1,2,:],ex2irfs.a[1,2,:]],
    title  = "IRFs to z shock");

plot(p1,p2, layout = (1,2),
    xlabel     = "Time",
    label      = ["Agg. Demand (q)" "Price (p)"],
    xlim       = (1,ex2irfs.T),
    lw         = 3,
    legend     = :bottomright,
    legendfont = font(10),
    tickfont   = font(10),
    framestyle = :box)

In [ ]:
p1 = plot(1:ex2irfs.T,ex2irfs.x[1,1,:]-ex2irfs.a[1,1,:],
    title  = "Output (q->y)");

p2 = plot(1:ex2irfs.T,[ex2irfs.a[1,1,1];ex2irfs.a[1,1,2:end]-ex2irfs.a[1,1,1:end-1]],
    title  = "Inflation (q->pi)")

p3 = plot(1:ex2irfs.T,ex2irfs.x[1,2,:]-ex2irfs.a[1,2,:],
    title  = "Output (z->y)");

p4 = plot(1:ex2irfs.T,[ex2irfs.a[1,2,1];ex2irfs.a[1,2,2:end]-ex2irfs.a[1,2,1:end-1]],
    title  = "Inflation (z->pi)")

plot(p1,p2,p3,p4, layout = (2,2),
    xlim       = (1,ex2irfs.T),
    lw         = 3,
    legend     = false,
    tickfont   = font(12),
    framestyle = :box)
